In [ ]:
import cv2
import mediapipe as mp
import time

class FaceMeshDetector:
    def __init__(self):
        self.mpDraw = mp.solutions.drawing_utils
        self.mpFaceMesh = mp.solutions.face_mesh
        self.faceMesh = self.mpFaceMesh.FaceMesh(max_num_faces=2)
        self.drawSpec = self.mpDraw.DrawingSpec(thickness=1, circle_radius=2)

        
        self.FACE_CONNECTIONS = [
            (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8),
            (8, 9), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16),
            (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24),
            (24, 25), (26, 27), (27, 28), (28, 29), (29, 30), (30, 31), (31, 32),
            (32, 33), (33, 34), (34, 35), (36, 37), (37, 38), (38, 39), (39, 40),
            (40, 41), (41, 36), (42, 43), (43, 44), (44, 45), (45, 46), (46, 47),
            (47, 42), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54),
            (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 48),
        ]

    def process_frame(self, img):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.faceMesh.process(imgRGB)
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                self.mpDraw.draw_landmarks(img, faceLms, self.FACE_CONNECTIONS,
                                           self.drawSpec, self.drawSpec)
                for id, lm in enumerate(faceLms.landmark):
                    ih, iw, ic = img.shape
                    x, y = int(lm.x * iw), int(lm.y * ih)
                    if 36 <= id <= 47:  # Check if the landmark is in the range of eyes
                        cv2.circle(img, (x, y), 10, (0, 0, 255), cv2.FILLED)  # Larger circle for eyes (red)
                    else:
                        cv2.circle(img, (x, y), 5, (0, 0, 255), cv2.FILLED)  

        return img

    def run_webcam(self):
        cap = cv2.VideoCapture(0)
        pTime = 0

        while True:
            success, img = cap.read()

            if not success:
                print("Error reading frame from webcam")
                break

            img = self.process_frame(img)

            cTime = time.time()
            fps = 1 / (cTime - pTime)
            pTime = cTime

            cv2.putText(img, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_PLAIN,
                        3, (0, 0, 255), 3)  
            cv2.imshow("Image", img)

            if cv2.waitKey(1) & 0xFF == 27: 
                break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    detector = FaceMeshDetector()
    detector.run_webcam()
